In [ ]:
from IPython.display import display, HTML
display(HTML("""
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:300px;
    overflow:auto;
}
</style>
"""))


# 데이터 엔지니어 포트폴리오
### 미니 경진대회 프로젝트 정리

---

**이름**: 양다은<br>
**이메일**: ekdmsl9701@gmail.com<br>
**GitHub**: https://github.com/dyang-Y  

---


## 소개
본 포트폴리오는 데이터 분석 및 머신러닝 프로젝트 경험을 기반으로, 실제 문제 해결 과정을 정리한 자료입니다.  
이어드림스쿨에서 자체적으로 진행한 미니 경진대회에서 제출한 코드와 분석 과정을 중심으로, **데이터 탐색**, **전처리**, **모델링**, **성능 평가**를 체계적으로 기록하였습니다.  

---

## 문서 구성
본 문서는 다음과 같은 순서로 구성되어 있습니다.

1. **프로젝트 개요**   

2. **데이터 탐색 및 전처리**  

3. **모델링 및 성능 평가**  

5. **회고 및 배운 점**  

---

## 목적
위와 같은 구성을 통해 단순한 코드 작성 능력뿐만 아니라, 문제 해결 과정 전반에 대한 제 역량을 보여드리고자 합니다.

---



# 미니 경진대회 - 선박 도장 품질 분류

선박 도장의 품질은 선체의 내구성과 안전에 직접적인 영향을 미치는 중요한 요소입니다. 하지만 기존의 품질 검사는 주로 수작업으로 이루어져 시간과 비용이 많이 소요되며, 검사원의 숙련도에 따라 결과가 달라지는 한계가 있었습니다.

본 프로젝트는 **딥러닝 이미지 분류 모델**을 활용하여 선박 도장 이미지를 자동으로 분류하고, 이를 통해 검사 과정을 자동화하여 효율성과 객관성을 높이는 것을 목표로 했으며, 최종적으로는 경진대회 상위권의 성능을 내는 모델을 개발하고자 했습니다.

## 라이브러리 설치 및 로드


In [ ]:
!pip install -U scikit-learn pandas numpy Pillow
!pip install lightgbm
!pip install tensorflow
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.2-c


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# 데이터 처리 및 분석
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
# 머신러닝 모델링
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import lightgbm as lgb
import tensorflow as tf

# 평가 및 학습 보조
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tqdm import tqdm

# 경로 설정 및 기본 변수
ROOT = Path("/mnt/elice/dataset")
SUBMIT_DIR = ROOT / "제출용 데이터"

2025-08-20 03:30:45.613565: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-20 03:30:45.630204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755660645.648930   17791 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755660645.655252   17791 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-20 03:30:45.674500: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## 데이터 탐색

본 분석에는 선박 도장 상태를 담은 이미지 데이터가 사용되었습니다.

#### **데이터 구조**
- **Train/Test 데이터:** 학습 및 테스트용 이미지가 별도 폴더에 제공되고 있습니다.
- **Class:** 이미지는 손상 종류에 따라 아래와 같이 총 6개의 폴더로 분류되어 있습니다.
    * TS_도막 손상_도막떨어짐
    * TS_도막 손상_스크래치
    * TS_도장 불량_부풀음
    * TS_도장 불량_이물질포함
    * TS_양품_선수
    * TS_양품_외판

#### **Label**
각 폴더명은 다음과 같은 손상 종류에 해당합니다.
* TS_도막 손상_도막떨어짐 -> **도막떨어짐**
* TS_도막 손상_스크래치 -> **스크래치**
* TS_도장 불량_부풀음 -> **부풀음**
* TS_도장 불량_이물질포함 -> **이물질포함**
* TS_양품_선수 -> **양품**
* TS_양품_외판 -> **양품**


In [ ]:
# Label 정의: 폴더 이름을 실제 라벨로 매핑
folder_to_label = {
    "TS_도막 손상_스크래치": "스크래치",
    "TS_도장 불량_부풀음": "부풀음",
    "TS_도막 손상_도막떨어짐": "도막떨어짐",
    "TS_도장 불량_이물질포함": "이물질포함",
    "TS_양품_선수": "양품",
    "TS_양품_외판": "양품",
}

# 이미지 처리 관련 기본 설정
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp"}
IMG_SIZE = (128, 128)

탐색한 이미지 데이터를 머신러닝 모델이 학습할 수 있는 숫자 형태의 벡터로 변환합니다. 이 과정에서는 사전 학습된 VGG16 모델을 특징 추출기로 사용하여, 이미 검증된 모델의 지식을 활용함으로써, 빠른 개발 속도와 높은 분류 정확도를 동시에 달성하고자 했습니다.

본격적인 처리에 앞서 코드의 안정성을 위해 데이터의 존재 여부를 먼저 확인하고, 이어서 특징 추출에 사용할 VGG16 모델을 준비했습니다.

In [ ]:
# 디버깅
print("\n[디버깅] 각 폴더의 존재 여부와 파일 수를 확인합니다.")
found_any_images = False
for folder, label in folder_to_label.items():
    d = ROOT / folder
    if not d.exists():
        print(f"경고: '{d}' 폴더를 찾을 수 없습니다. 경로를 확인하세요.")
    else:
        image_files = [p for p in d.rglob("*") if p.suffix.lower() in IMG_EXTS]
        if not image_files:
            print(f"경고: '{d}' 폴더는 존재하지만, 이미지 파일이 없습니다.")
        else:
            print(f"확인: '{d}' 폴더에서 {len(image_files)}개의 이미지 파일을 찾았습니다.")
            found_any_images = True

if not found_any_images:
    print("\n[치명적 오류] 어떤 폴더에서도 이미지 파일을 찾지 못했습니다. 스크립트를 중단하기 전에 경로를 수정해주세요.")


print("\n데이터 로딩 및 CNN 특징 추출을 시작")


[디버깅] 각 폴더의 존재 여부와 파일 수를 확인합니다.
확인: '/mnt/elice/dataset/TS_도막 손상_스크래치' 폴더에서 200개의 이미지 파일을 찾았습니다.
확인: '/mnt/elice/dataset/TS_도장 불량_부풀음' 폴더에서 200개의 이미지 파일을 찾았습니다.
확인: '/mnt/elice/dataset/TS_도막 손상_도막떨어짐' 폴더에서 200개의 이미지 파일을 찾았습니다.
확인: '/mnt/elice/dataset/TS_도장 불량_이물질포함' 폴더에서 200개의 이미지 파일을 찾았습니다.
확인: '/mnt/elice/dataset/TS_양품_선수' 폴더에서 200개의 이미지 파일을 찾았습니다.
확인: '/mnt/elice/dataset/TS_양품_외판' 폴더에서 200개의 이미지 파일을 찾았습니다.

데이터 로딩 및 CNN 특징 추출을 시작


In [ ]:
# 사전 학습된 VGG16 모델 로드

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3), pooling='avg')
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

I0000 00:00:1755660648.889097   17791 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8108 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe MIG 1g.10gb, pci bus id: 0000:e3:00.0, compute capability: 8.0


### 특징 추출

수집된 이미지 데이터는 픽셀의 집합으로, 머신러닝 모델이 직접 학습하기 어렵습니다. 따라서 사전 학습된 VGG16 모델을 특징 추출기로 사용하여, 각 이미지를 512차원의 숫자 벡터로 변환하는 작업을 수행했습니다.

**주요 처리 과정은 다음과 같습니다**
1.  정의된 각 클래스 폴더를 순회하며 이미지 파일을 탐색합니다.
2.  모든 이미지를 동일한 크기(128x128)와 색상 채널(RGB)로 통일합니다.
3.  VGG16 모델이 학습된 방식과 동일하게 픽셀 값을 전처리합니다.
4.  전처리된 이미지를 VGG16 모델에 입력하여, 이미지의 핵심 특징을 담은 특징 벡터를 추출합니다.
5.  추출된 특징 벡터는 `X_features`에, 해당 이미지의 정답 라벨은 `y`에 저장하여 모델 학습을 위한 데이터셋을 구축합니다.

마지막으로, 모든 과정이 끝난 후 이미지 로딩이 정상적으로 이루어졌는지 다시 한번 확인하는 안전장치를 추가하여 코드의 안정성을 높였습니다.

In [7]:
X_features, y = [], []
for folder, label in tqdm(folder_to_label.items(), desc="진행률"):
    d = ROOT / folder
    
    if not d.exists():
        print(f"\n[경고] '{d}' 폴더를 찾을 수 없습니다. folder_to_label 딕셔너리의 폴더 이름이나 ROOT 경로 확인 필요")
        continue
        
    for p in d.rglob("*"):
        if p.suffix.lower() in IMG_EXTS:
            try:
                im = Image.open(p).convert("RGB").resize(IMG_SIZE)
                arr = np.array(im, dtype=np.float32)
                
                arr_expanded = np.expand_dims(arr, axis=0)
                arr_preprocessed = preprocess_input(arr_expanded)
                features = feature_extractor.predict(arr_preprocessed, verbose=0)
                
                X_features.append(features.flatten())
                y.append(label)
            except Exception as e:
                    print(f"Error: {p} 파일 처리 중 문제 발생 - {e}")
                    
print(f"특징 불러오기 완료 - {len(X_features)}개의 이미지에서 특징 추출")

진행률:   0%|          | 0/6 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1755660651.056279   17829 service.cc:148] XLA service 0x7f8214017910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755660651.056304   17829 service.cc:156]   StreamExecutor device (0): NVIDIA A100 80GB PCIe MIG 1g.10gb, Compute Capability 8.0
2025-08-20 03:30:51.065844: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1755660651.121990   17829 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1755660653.077696   17829 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
진행률: 100%|██████████| 6/6 [03:03<00:00, 30.58s/it]

특징 불러오기 완료 - 1200개의 이미지에서 특징 추출


In [8]:
if not X_features:
    raise ValueError("이미지를 하나도 불러오지 못했습니다. 경로와 폴더 이름을 확인해주세요.")

print("\n모델 학습을 위한 데이터 준비")


모델 학습을 위한 데이터 준비


## 모델링

이미지에서 추출한 특징을 기반으로 최종 분류를 수행하기 위해 **LightGBM 모델**을 사용했습니다.
LightGBM은 대용량 데이터에서도 학습 속도가 빠르고 높은 성능을 보이는 장점이 있어 최종 모델로 선정하게 되었습니다.

In [ ]:
X_features = np.array(X_features)
y = np.array(y)

머신러닝 모델은 '스크래치', '부풀음'과 같은 문자열 형태의 라벨을 직접 이해할 수 없습니다. 따라서, 각 문자열 라벨을 고유한 숫자(예: 스크래치 -> 0, 부풀음 -> 1)로 변환해주는 라벨 **인코딩 과정**을 수행합니다.

In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

#### 학습/검증 데이터 분리

전체 데이터를 학습용과 검증용으로 분리합니다. 모델을 학습용 데이터로만 훈련시킨 뒤, 한 번도 본 적 없는 검증용 데이터로 성능을 평가함으로써, 모델이 과적합되지 않았는지 객관적으로 확인할 수 있습니다.

특히, `stratify=y_encoded` 옵션을 사용하여 학습용과 검증용 데이터에 포함된 각 라벨의 비율이 원본 데이터와 동일하게 유지되도록 했습니다. 이는 데이터 불균형 문제로 인한 성능 왜곡을 방지하는 중요한 과정입니다.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_features, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print("Data 준비 완료")
print(f"학습 Data: {X_train.shape}, {y_train.shape}")
print(f"검증 Data: {X_val.shape}, {y_val.shape}")

Data 준비 완료
학습 Data: (960, 512), (960,)
검증 Data: (240, 512), (240,)


### LightGBM 모델 학습

이제 준비된 학습 데이터셋을 사용하여 LightGBM 모델을 본격적으로 학습시킵니다.

모델이 검증 데이터셋의 성능을 계속 모니터링하다가, 성능이 더 이상 좋아지지 않으면 (100번 반복 후) 자동으로 학습을 중단하는 조기 종료 기법을 적용했습니다. 이를 통해 불필요한 추가 학습을 방지하고 과적합의 위험을 줄였습니다.

In [10]:
clf = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=len(le.classes_),
    n_estimators=1000,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1,
)
clf.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='logloss',
        callbacks=[lgb.early_stopping(100, verbose=True)])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011812 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 59475
[LightGBM] [Info] Number of data points in the train set: 960, number of used features: 511
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[Lig

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.05
,n_estimators,1000
,subsample_for_bin,200000
,objective,'multiclass'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


## 성능 평가
이제 학습된 모델의 성능을 검증 데이터셋을 통해 확인합니다.

In [ ]:
y_pred_val = clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred_val)
print(f"model 정확도: {accuracy:.4f}")

model 정확도: 0.9667


/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


## 최종 결과 및 제출 파일 생성

학습된 모델의 성능을 평가하기 위해 정확도를 주요 지표로 사용했습니다. 검증 데이터셋으로 평가한 결과, **약 96.7%의 분류 정확도**를 달성했습니다.

* 아래 데이터프레임은 예시입니다.
   
|      | label |
| --------- | ----- |
| 00001.jpg | 양품    |
| 00002.jpg | 스크래치    |
| 00003.jpg | 양품    |
| 00004.jpg | 도막떨어짐    |
| 00005.jpg | 양품    |

### 테스트 데이터에 대한 예측 수행

검증 데이터셋에서 96.7%의 높은 성능을 보인 모델을 사용하여, 한 번도 학습에 사용되지 않은 테스트 데이터의 손상 종류를 예측합니다.

처리 과정은 학습 데이터를 처리했을 때와 마찬가지로, 각 테스트 이미지를 불러와 VGG16 특징 추출기를 거친 뒤, 학습된 LightGBM 모델로 최종 라벨을 예측합니다.

In [ ]:
filenames, labels = [], []
for p in tqdm(list(SUBMIT_DIR.rglob("*")), desc="파일 생성중"):
    if p.is_file() and p.suffix.lower() in IMG_EXTS:
        im = Image.open(p).convert("RGB").resize(IMG_SIZE)
        arr = np.array(im, dtype=np.float32)
        arr_expanded = np.expand_dims(arr, axis=0)
        arr_preprocessed = preprocess_input(arr_expanded)
        
        features = feature_extractor.predict(arr_preprocessed, verbose=0)
        y_pred = clf.predict(features)
        label_ko = le.inverse_transform(y_pred)[0]
        
        filenames.append(p.name)
        labels.append(label_ko)

파일 생성중:   0%|          | 0/1000 [00:00<?, ?it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:   0%|          | 1/1000 [00:00<02:35,  6.43it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:   0%|          | 2/1000 [00:00<02:34,  6.46it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:   0%|          | 3/1000 [00:00<02:31,  6.57it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  war

파일 생성중:   6%|▋         | 64/1000 [00:09<02:17,  6.81it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:   6%|▋         | 65/1000 [00:09<02:24,  6.45it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:   7%|▋         | 66/1000 [00:09<02:20,  6.66it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:   7%|▋         | 67/1000 [00:09<02:19,  6.70it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature

파일 생성중:  13%|█▎        | 128/1000 [00:18<02:03,  7.08it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  13%|█▎        | 129/1000 [00:18<01:56,  7.46it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  13%|█▎        | 130/1000 [00:18<01:59,  7.29it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  13%|█▎        | 131/1000 [00:18<01:55,  7.53it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with fea

파일 생성중:  16%|█▌        | 160/1000 [00:22<02:05,  6.70it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  16%|█▌        | 161/1000 [00:23<02:04,  6.72it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  16%|█▌        | 162/1000 [00:23<02:14,  6.23it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  16%|█▋        | 163/1000 [00:23<02:02,  6.85it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with fea

파일 생성중:  19%|█▉        | 192/1000 [00:27<01:55,  7.01it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  19%|█▉        | 193/1000 [00:27<01:46,  7.56it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  19%|█▉        | 194/1000 [00:27<01:49,  7.39it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  20%|█▉        | 195/1000 [00:27<01:51,  7.25it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with fea

파일 생성중:  22%|██▏       | 224/1000 [00:32<01:47,  7.20it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  22%|██▎       | 225/1000 [00:32<01:48,  7.13it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  23%|██▎       | 226/1000 [00:32<01:49,  7.09it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
파일 생성중:  23%|██▎       | 227/1000 [00:32<01:53,  6.82it/s]/home/elicer/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with fea

### 제출 파일 생성

예측된 결과를 경진대회 규격에 맞는 `submission.csv` 파일 형태로 만들었습니다. 파일명과 예측된 라벨로 구성된 데이터프레임을 생성하여 저장합니다.

In [ ]:
df_submit = pd.DataFrame({"label": labels}, index=filenames)
df_submit = df_submit.sort_index() 
df_submit.to_csv("./submission.csv", encoding="utf-8")

df_submit.head()


---

## 회고 및 배운 점

### 프로젝트 요약
본 프로젝트를 통해 선박 도장 이미지 데이터를 분석하고, VGG16과 LightGBM 모델을 활용하여 약 96.7%의 정확도로 도장 품질을 분류하는 모델을 성공적으로 구축했습니다. 이를 통해 기존 수작업 검사 방식의 비효율성을 개선할 수 있는 자동화의 가능성을 확인했습니다.

### 어려웠던 점 및 해결 과정
- **어려웠던 점:** 정확도와 과적합 사이의 딜레마  
    모델 학습 초기, 간단한 모델을 사용했을 때는 성능이 너무 낮고, 반대로 복잡한 모델을 사용해 학습 데이터에 대한 정확도를 높였더니 검증 데이터에서 성능이 떨어지는 전형적인 과적합 현상이 발생했습니다. 단순히 한 모델의 파라미터를 조정하는 것만으로는 두 문제를 동시에 해결하기 어려운 딜레마에 부딪혔습니다.
- **해결 과정:** 체계적인 모델 비교를 통한 최적 모델 선정  
    이러한 트레이드오프 관계를 해결하기 위해, 접근 방식을 '하나의 모델 최적화'에서 '주어진 문제에 가장 적합한 모델을 찾는 과정'으로 변경했습니다. VGG16으로 추출한 특징 벡터를 가지고 로지스틱 회귀, 랜덤 포레스트 등 여러 모델을 테스트했습니다.
    그 결과, 복잡도는 유지하면서도 뛰어난 규제 기능으로 과적합을 억제하는 LightGBM 모델이 데이터셋에 가장 적합하다고 판단했습니다. 최종적으로 LightGBM을 채택하여 과적합을 방지하면서도 96.7%라는 높은 검증 정확도를 달성할 수 있었습니다.

### 배운 점 및 향후 개선 방안
- **배운 점:** 균형 잡힌 모델 평가와 유연한 사고의 중요성  
    이번 프로젝트를 통해 좋은 모델이란 단순히 정확도만 높은 모델이 아니라, 과적합 등 다양한 요소를 종합적으로 고려하여 균형을 맞추는 것이 중요함을 깨달았습니다. 또한, 하나의 접근 방식이 막혔을 때 다양한 모델을 비교하고 탐색하는 유연한 문제 해결 방식이 어떻게 더 좋은 결과로 이어질 수 있는지 체감할 수 있었습니다.
- **개선 방안:** 미세 조정을 통한 성능 극대화  
    현재는 VGG16 모델을 고정된 특징 추출기로만 사용하여 좋은 성능을 얻었지만, 여기서 더 나아가 모델의 일부 레이어까지 재학습시키는 미세 조정 기법을 적용한다면, 데이터셋의 특징을 더 깊이 학습하여 정확도를 한계까지 끌어올릴 수 있지 않았을까 라는 생각이 듭니다.